In [1]:
import os
import json
import numpy as np
import pandas as pd
from utils.eval import *
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
a = 45; b= 140
mode = "bi"
w_sec = 5; h_sec = w_sec/2


jdata1 = evaluation_single(a, b, mode, "anchor_zero", tolerance=0.13)    # overall acc1 per body segment
jdata2 = evaluation_single(a, b, mode, "anchor_peak", tolerance=0.13)    # overall acc1 per body segment
jdata3 = evaluation_single(a, b, mode, "anchor_energy",tolerance=0.13)    # overall acc1 per body segment

In [ ]:

print("zero-vel")
for key in jdata1["bpm_median"].keys():
    print(key,"---", jdata1["bpm_median"][key]['accuracy'])
    
print("\npeak-vel")
for key in jdata2["bpm_median"].keys():
    print(key,"---", jdata2["bpm_median"][key]['accuracy'])
    
print("\nenergy")
for key in jdata3["bpm_median"].keys():
    print(key,"---", jdata3["bpm_median"][key]['accuracy'])    

### Proposed method

In [2]:
a, b = 45, 140

accuracy_zero = evaluation_multi_segment("anchor_zero", "uni",a,b, tolerance=0.13)
accuracy_peak= evaluation_multi_segment("anchor_peak", "uni",a,b, tolerance=0.13)
accuracy_energy = evaluation_multi_segment("anchor_energy", "uni",a,b, tolerance=0.13)


In [3]:
hit_genres_multi = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_genres']
hit_segment_names = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_seg_names']


In [ ]:
print("uni")
print("anchor_zero")

for key in accuracy_zero.keys():
    print(key,"---", round(accuracy_zero[key]['acc'],2))
    
print("\nanchor_peak")
for key in accuracy_peak.keys():
    print(key,"---", round(accuracy_peak[key]['acc'],2))    

print("\naccuracy_energy")
for key in accuracy_energy.keys():
    print(key,"---", round(accuracy_energy[key]['acc'],2))    

## Evaluation: best of N strategy

In [ ]:
segment_names = [["both_hand_y", "both_foot_y"],
                ["both_hand_y", "both_foot_y", "torso_y"]]

a, b = 45, 140
mode = "uni"

json_data = eval_best_of_n(segment_names, a, b, mode, "anchor_zero")

chosen = json_data['both_hand_y_both_foot_y_torso_y']['chosen_candidate_bpm']
hit_genres_id = json_data['both_hand_y_both_foot_y_torso_y']['hit_genres']

# hit_idx = json_data['both_hand_y_both_foot_y_torso_y']['hit_idx']
# hit_ref_bpm = json_data['both_hand_y_both_foot_y_torso_y']['hit_ref_bpm']
# df_idx = load_pickle("./tempo_estimation_output/tempo_45_140/multi/anchor_zero/bothhand_y_bothfoot_y_torso_y_uni.pkl")
# gtempo = df_idx["gtempo"].to_numpy() # --- can use hit_idx
# hit_bpm = gtempo[hit_idx]


## Octave Search

In [15]:
hit_idx = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_idx']
hit_ref_bpm = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_ref_bpm']
hit_segment_names = accuracy_zero['bothhand_y_bothfoot_y_torso_y']['hit_seg_names']

df_idx = load_pickle("./tempo_estimation_output/tempo_45_140/multi/anchor_zero/bothhand_y_bothfoot_y_torso_y_uni.pkl")
gtempo = df_idx["gtempo"].to_numpy() # --- can use hit_idx
hit_bpm = gtempo[hit_idx]

In [17]:
from collections import Counter

tolerance = 0.1  # ±3.5%
half_mask = np.abs(hit_bpm - 0.5 * hit_ref_bpm) <= (tolerance * hit_ref_bpm)
double_mask = np.abs(hit_bpm - 2.0 * hit_ref_bpm) <= (tolerance * hit_ref_bpm)
ref_mask = np.abs(hit_bpm - hit_ref_bpm) <= (tolerance * hit_ref_bpm)

# Counts
half_count = np.sum(half_mask)
double_count = np.sum(double_mask)
ref_count = np.sum(ref_mask)
total = len(hit_bpm)

print("total hits:", total)
print(f"Half-time matches:  {half_count} ({half_count/total:.2%})")
print(f"Double-time matches:{double_count} ({double_count/total:.2%})")
print(f"Reference matches:  {ref_count} ({ref_count/total:.2%})")

# --- Dominant segment for half-time ---
if half_count > 0:
    half_segments = hit_segment_names[half_mask]
    dominant_segment, count = Counter(half_segments).most_common(1)[0]
    print(f"Dominant segment for half-time matches: {dominant_segment} ({count} occurrences)")
else:
    print("No half-time matches found.")


total hits: 978
Half-time matches:  150 (15.34%)
Double-time matches:0 (0.00%)
Reference matches:  828 (84.66%)
Dominant segment for half-time matches: torso_y (115 occurrences)


## Body Part Contribution per Dance Genre

In [ ]:
# part_counts = find_body_contribution_best_of_n(chosen, hit_genres_id)
part_counts = find_body_contribution_multi(hit_genres_multi, hit_segment_names)


In [ ]:
genre_order = ['House', 'Middle Hip-hop', 'Break','LA style Hip-hop',
				'Krump', 'Pop','Lock', 'Waack', 'Ballet Jazz', 'Street Jazz']

part_counts["genre"] = pd.Categorical(
    part_counts["genre"],
    categories=genre_order,
    ordered=True
    )

plt.figure(figsize=(10, 6), dpi=80)
sns.barplot(
    data=part_counts,
    x="percentage",
    y="genre",
    hue="body_part",
    dodge=True,
    order=genre_order,
)

label_map = {
    "both_hand_y": "Hands (y)",
    "both_foot_y": "Feet (y)",
    "torso_y": "Torso (y)"
}
handles, labels = plt.gca().get_legend_handles_labels()
new_labels = [label_map.get(lbl, lbl) for lbl in labels]

# plt.title("Body Part Contribution per Dance Genre")
plt.xlabel("Percentage (%)")
plt.ylabel("Dance Genre")
plt.legend(handles, new_labels,title="Body Part",  loc="lower right")
plt.tight_layout()
plt.show()

